In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import das dependencias
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [3]:
#definição dos modelos a serem usados da OPEN AI
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)

In [21]:
#Carregamento do PDF para o rag
pdf = "DOC-SF238339076816-20230503.pdf"
pdf_loader = PyPDFLoader(pdf, extract_images=False)
pages = pdf_loader.load_and_split()

In [22]:
#Separar o arquivo em chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20, length_function=len, add_start_index=True)

chunks = text_splitter.split_documents(pages)

In [23]:
#Salvar no banco vetorial

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="vectors")

db.persist()

In [24]:
#Carregar os vetores
vectordb = Chroma(persist_directory="vectors", embedding_function=embeddings_model)

#load retriever - buscar chunks relevantes
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

#cadeia de prompt para a chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [31]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = chain({"input_documents": context, "question": question}, return_only_outputs=True)["output_text"]
    return answer, context



In [32]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

User:  Do que se trata o capítulo IX?


Answer:  O Capítulo IX trata da "rastreabilidade das decisões durante o ciclo de vida de sistemas de inteligência artificial como meio de prestação de contas e atribuição de responsabilidades a uma pessoa natural ou jurídica".


In [33]:
print(context[0])

page_content='VIII – a privacidade, a proteção de dados e a autodeterminação 
informativa; 
IX – a promoção da pesquisa e do desenvolvimento com a 
finalidade de estimular a inovação nos setores produtivos e no poder público; e 
X – o acesso à informação e à educação, e a conscientização sobre 
os sistemas de inteligência artificial e suas aplicações. 
Art. 3º O desenvolvimento, a implementação e o uso de sistemas 
de inteligência artificial observarão a boa-fé e os seguintes princípios: 
I – crescimento inclusivo, desenvolvimento sustentável e bem-
estar; 
II – autodeterminação e liberdade de decisão e de escolha; 
III – participação humana no ciclo da inteligência artificial e 
supervisão humana efetiva; 
IV – não discriminação; 
V – justiça, equidade e inclusão; 
VI – transparência, explicabilidade, inteligibilidade e 
auditabilidade; 
VII – confiabilidade e robustez dos sistemas de inteligência 
artificial e segurança da informação; 
VIII – devido processo legal, contestabilidade e